# PyCity Schools Analysis

- The school district displays significant variation in performance, with the highest performing school achieving an impressive overall pass rate of 91% among students, while the lowest performing school records a much lower pass rate of only 53%.

- Charter schools, which are independent publicly funded schools, outperform district schools in terms of student performance.

- The difference in performance between charter schools and district schools appears to be associated with the size of the schools.

- Smaller schools consistently exhibit considerably better performance outcomes compared to larger schools within the district.

- Contrary to common expectations, an increase in spending per student does not appear to have a positive impact on academic performance. In fact, there may be a correlation suggesting that increased spending per student is linked to decreased test scores.

- Further analysis is necessary to determine the significance of this negative correlation and whether it truly indicates a causal relationship between spending and academic performance.

--- 

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load 
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

In [2]:
# Calculate number of schools in the district
school_count = school_data_complete["school_name"].nunique()

In [3]:
# Calculuate number of students in the district
student_count = school_data_complete["student_name"].count()

In [4]:
# Calculate total budget
total_budget = school_data["budget"].sum()


In [5]:
# Calculate district-wide average math score
average_math_score = (school_data_complete["math_score"].sum()/student_count)

In [6]:
# Calculate district-wide average reading score
average_reading_score = (school_data_complete["reading_score"].sum()/student_count)

In [7]:
# Calculate the percent of students who passed math
pass_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
pass_math_percent = pass_math_count / float(student_count) * 100

In [8]:
# Calculate the percent of students who passed reading
pass_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
pass_reading_percent = pass_reading_count / float(student_count) * 100

In [9]:
# Calculate the percent of sutdents who passed both math and reading
pass_overall_count = school_data_complete[(school_data_complete["math_score"] >= 70) 
                    & (school_data_complete["reading_score"] >= 70)].count()["student_name"]
pass_overall_percent = pass_overall_count / float(student_count) * 100

In [36]:
# Create a dataframe with district summary statistics
district_summary = pd.DataFrame([{"Total Schools": school_count, "Total Students": student_count, 
                                "Total Budget": total_budget, "Average Math Score": average_math_score, 
                                "Average Reading Score": average_reading_score, "% Passing Math": pass_math_percent, 
                                "% Passing Reading": pass_reading_percent, "% Passing Overall": pass_overall_percent}])
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                "% Passing Reading", "% Passing Overall"]] = district_summary[["Average Math Score", "Average Reading Score", 
                                                            "% Passing Math", "% Passing Reading", "% Passing Overall"]].round(2)

# Display the dataframe
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,65.17


## School Summary

In [11]:
# Start datafram for school summary from school_data. Remove school id
school_data = school_data.drop("School ID", axis = 1)

In [12]:
# Calculate per student budget and add to dataframe
school_data["Per Student Budget"] = school_data["budget"]/school_data["size"]

In [13]:
# Calculate average math and reading scores for each school
average_scores_school = pd.DataFrame(school_data_complete.groupby("school_name")[["math_score", "reading_score"]].mean())

In [14]:
# Add Average scores to School Summary Dataframe
school_summary = pd.merge(school_data, average_scores_school, on = "school_name", how = "outer")

In [15]:
# Calculate percent passing math for each school

# Count number of passing students per school
count_passing_math_school = (school_data_complete[(school_data_complete["math_score"] >= 
                            70)].groupby("school_name").count()["student_name"])

# Create temporary dataframe to store number of passing students and total students per school
percent_passing_math_school = pd.merge(count_passing_math_school, school_data[['school_name','size']],on='school_name', how='left')

# Calculate percent of students passing math
percent_passing_math_school["% Passing Math"] = (percent_passing_math_school["student_name"]/
                                                percent_passing_math_school["size"] *100)


In [16]:
# Add "% Passing Math" to School Summary
school_summary = pd.merge(school_summary, percent_passing_math_school[["school_name","% Passing Math"]], on = "school_name", how= "left")

In [17]:
# Calculate percent passing reading for each school

# Count number of passing students per school
count_passing_reading_school = (school_data_complete[(school_data_complete["reading_score"] >= 
                            70)].groupby("school_name").count()["student_name"])

# Create temporary dataframe to store number of passing students and total students per school
percent_passing_reading_school = pd.merge(count_passing_reading_school, school_data[['school_name','size']],on='school_name', how='left')

# Calculate percent of students passing math
percent_passing_reading_school["% Passing Reading"] = (percent_passing_reading_school["student_name"]/
                                                        percent_passing_reading_school["size"] *100)

In [18]:
# Add "% Passing Reading" to School Summary
school_summary = pd.merge(school_summary, percent_passing_reading_school[["school_name","% Passing Reading"]], on = "school_name", how= "left")

In [19]:
# Calculate percent passing math and reading for each school

# Count number of passing students per school
count_passing_overall_school = (school_data_complete[(school_data_complete["math_score"] >= 
                            70) & (school_data_complete["reading_score"] >= 
                            70)].groupby("school_name").count()["student_name"])

# Create temporary dataframe to store number of passing students and total students per school
percent_passing_overall_school = pd.merge(count_passing_overall_school, school_data[['school_name','size']],on='school_name', how='left')

# Calculate percent of students passing math
percent_passing_overall_school["% Passing Overall"] = (percent_passing_overall_school["student_name"]/
                                                        percent_passing_overall_school["size"] *100)

In [20]:
# Add "% Passing Overall" to School Summary
school_summary = pd.merge(school_summary, percent_passing_overall_school[["school_name","% Passing Overall"]], on = "school_name", how= "left")

In [21]:
# Formatting
school_summary = school_summary.rename(columns = {"school_name": "School Name", "type": "School Type", "size": "Total Students", 
                "budget": "Total School Budget", "math_score": "Average Math Score", "reading_score": "Average Reading Score"})
school_summary = school_summary.sort_values(by = "School Name", ascending = True)
school_summary= school_summary.set_index("School Name")
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].astype(int).map(("${:,}".format))
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].astype(int).map("${:,}".format)
school_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                "% Passing Reading", "% Passing Overall"]] = school_summary[["Average Math Score", "Average Reading Score", 
                                                            "% Passing Math", "% Passing Reading", "% Passing Overall"]].round(2)
school_summary.index.name = None

# Display the dataframe
school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


## Highest Performing Schools (By % Passing Overall)

In [22]:
# Sort school summary by % overall passing highest to lowest
top_schools = school_summary.sort_values(["% Passing Overall"], ascending = False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


## Lowest Performing Schools (By % Passing Overall)

In [23]:
# Sort school summary by percent passing overall lowest to highest
bottom_schools = school_summary.sort_values(["% Passing Overall"], ascending = True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54


## Math Scores By Grade

In [24]:
# Separate complete school data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grader_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame
math_scores_by_grade = pd.DataFrame({"9th Grade": ninth_grader_math_scores, "10th Grade": 
                        tenth_grader_math_scores, "11th Grade": eleventh_grader_math_scores, 
                        "12th Grade": twelfth_grader_math_scores})


# Minor data wrangling
math_scores_by_grade.index.name = None

# Round Scores
math_scores_by_grade[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = math_scores_by_grade[["9th Grade", 
                                                                                "10th Grade", "11th Grade", "12th Grade"]].round(2)

# Display the dataframe
math_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Scores By Grade

In [25]:
# Group separated school data by `school_name` and take the mean of the `reading_score` column for each.
ninth_grader_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()



# Combine each of the scores above into single DataFrame
reading_scores_by_grade = pd.DataFrame({"9th Grade": ninth_grader_reading_scores, "10th Grade": 
                        tenth_grader_reading_scores, "11th Grade": eleventh_grader_reading_scores, 
                        "12th Grade": twelfth_grader_reading_scores})


# Minor data wrangling
reading_scores_by_grade.index.name = None

# Round Scores
reading_scores_by_grade[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = reading_scores_by_grade[["9th Grade", 
                                                                                "10th Grade", "11th Grade", "12th Grade"]].round(2)

# Display the dataframe
reading_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores By School Spending

In [37]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = school_summary.copy()

# Change "Per Student Budget" back to float value
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace(",", "")
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace("$", "", regex = False)
school_spending_df["Per Student Budget"]= school_spending_df["Per Student Budget"].astype(float)

# Slice the data and place it into bins
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], 
                                                    spending_bins, labels=labels, 
                                                    include_lowest=True)

# Display the dataframe
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,Spending Ranges (Per Student)
Bailey High School,District,"4,976","$3,124,928",628.0,77.05,81.03,66.68,81.93,54.64,$585-630
Cabrera High School,Charter,"1,858","$1,081,356",582.0,83.06,83.98,94.13,97.04,91.33,<$585
Figueroa High School,District,"2,949","$1,884,411",639.0,76.71,81.16,65.99,80.74,53.20,$630-645
Ford High School,District,"2,739","$1,763,916",644.0,77.10,80.75,68.31,79.30,54.29,$630-645
Griffin High School,Charter,"1,468","$917,500",625.0,83.35,83.82,93.39,97.14,90.60,$585-630
Hernandez High School,District,"4,635","$3,022,020",652.0,77.29,80.93,66.75,80.86,53.53,$645-680
Holden High School,Charter,427,"$248,087",581.0,83.80,83.81,92.51,96.25,89.23,<$585
Huang High School,District,"2,917","$1,910,635",655.0,76.63,81.18,65.68,81.32,53.51,$645-680
Johnson High School,District,"4,761","$3,094,650",650.0,77.07,80.97,66.06,81.22,53.54,$645-680
Pena High School,Charter,962,"$585,858",609.0,83.84,84.04,94.59,95.95,90.54,$585-630


In [32]:
# Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
spending_passing_overall= school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Overall"].mean()

# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores, "Average Reading Score": spending_reading_scores, 
                    "% Passing Math":  spending_passing_math, "% Passing Reading": spending_passing_reading, 
                    "% Passing Overall" : spending_passing_overall})

# Round scores
spending_summary[["Average Math Score", "Average Reading Score", 
              "% Passing Math", "% Passing Reading", "% Passing Overall"]] = spending_summary[["Average Math Score", "Average Reading Score", 
                                                                                           "% Passing Math", "% Passing Reading", "% Passing Overall"]].round(2)
# Display the dataframe
spending_summary


,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [28]:
# Establish the bins
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create a copy of the school summary since it has the "Total Students" 
school_size_df = school_summary.copy()

# Change "Total Students" back to float value
school_size_df["Total Students"] = school_size_df["Total Students"].str.replace(",", "")
school_size_df["Total Students"]= school_size_df["Total Students"].astype(int)

# Slice the data and place it into bins
school_spending_df["School Size"] = pd.cut(school_size_df["Total Students"], 
                                                    size_bins, labels=size_labels, 
                                                    include_lowest=True)
# Display the dataframe
school_spending_df



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,Spending Ranges (Per Student),School Size
Bailey High School,District,"4,976","$3,124,928",628.0,77.05,81.03,66.68,81.93,54.64,$585-630,Large (2000-5000)
Cabrera High School,Charter,"1,858","$1,081,356",582.0,83.06,83.98,94.13,97.04,91.33,<$585,Medium (1000-2000)
Figueroa High School,District,"2,949","$1,884,411",639.0,76.71,81.16,65.99,80.74,53.20,$630-645,Large (2000-5000)
Ford High School,District,"2,739","$1,763,916",644.0,77.10,80.75,68.31,79.30,54.29,$630-645,Large (2000-5000)
Griffin High School,Charter,"1,468","$917,500",625.0,83.35,83.82,93.39,97.14,90.60,$585-630,Medium (1000-2000)
Hernandez High School,District,"4,635","$3,022,020",652.0,77.29,80.93,66.75,80.86,53.53,$645-680,Large (2000-5000)
Holden High School,Charter,427,"$248,087",581.0,83.80,83.81,92.51,96.25,89.23,<$585,Small (<1000)
Huang High School,District,"2,917","$1,910,635",655.0,76.63,81.18,65.68,81.32,53.51,$645-680,Large (2000-5000)
Johnson High School,District,"4,761","$3,094,650",650.0,77.07,80.97,66.06,81.22,53.54,$645-680,Large (2000-5000)
Pena High School,Charter,962,"$585,858",609.0,83.84,84.04,94.59,95.95,90.54,$585-630,Small (<1000)


In [35]:
# Calculate averages for the desired columns. 
size_math_scores = school_spending_df.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_spending_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_spending_df.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = school_spending_df.groupby(["School Size"])["% Passing Reading"].mean()
size_passing_overall = school_spending_df.groupby(["School Size"])["% Passing Overall"].mean()

# Assemble into DataFrame
size_summary = pd.DataFrame({"Average Math Score": size_math_scores, "Average Reading Score": size_reading_scores, 
                    "% Passing Math":  size_passing_math, "% Passing Reading": size_passing_reading, 
                    "% Passing Overall" : size_passing_overall})

# Round Scores
size_summary[["Average Math Score", "Average Reading Score", 
              "% Passing Math", "% Passing Reading", "% Passing Overall"]] = size_summary[["Average Math Score", "Average Reading Score", 
                                                                                           "% Passing Math", "% Passing Reading", "% Passing Overall"]].round(2)

# Display the dataframe
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.28


## Scores by School Type

In [34]:
# Group the school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_passing_overall_by_type = school_summary.groupby(["School Type"])["% Passing Overall"].mean()

# Assemble into dataframe
type_summary = pd.DataFrame({"Average Math Score": average_math_score_by_type, "Average Reading Score": 
                average_reading_score_by_type, "% Passing Math": average_percent_passing_math_by_type, 
                "% Passing Reading": average_percent_passing_reading_by_type, "% Passing Overall": average_percent_passing_overall_by_type})
# Round scores
type_summary[["Average Math Score", "Average Reading Score", 
              "% Passing Math", "% Passing Reading", "% Passing Overall"]] = type_summary[["Average Math Score", "Average Reading Score", 
                                                                                           "% Passing Math", "% Passing Reading", "% Passing Overall"]].round(2)
# Display the dataframe
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
